### How to set-up SentiStrenght?
1. Download data from here, using UvA VPN: http://sentistrength.wlv.ac.uk/index.html
2. UvA VPN you can download from here: https://student.uva.nl/en/content/az/uvavpn/download/download-uvavpn-software.html
3. Check whether you have a JAVA on your computer, it is neccessary to SentiStrenght work! In case not: https://www.java.com/pl/download/
4. Download jar file from here: http://sentistrength.wlv.ac.uk/jkpop/
5. If you want to see sentiment for other languages check that page: https://github.com/bobvdvelde/SentiStrength_for_python and that one: https://github.com/felipebravom/StaticTwitterSent/tree/master/extra/SentiStrength or just google for more :)


test check to see whether it works + set up of the paths

In [22]:
from sentistrength import PySentiStr
senti = PySentiStr()
senti.setSentiStrengthPath('C:/SentStrength_Data/SentiStrength.jar') # Note: Provide absolute path instead of relative path
senti.setSentiStrengthLanguageFolderPath('C:/SentStrength_Data/SentStrengthData') # Note: Provide absolute path instead of relative path
#senti = PySentiStr()

result = senti.getSentiment(['What a nice day',"What a lovely day",'What a bad day, how to live here now', 'Its okay'], score = 'scale')
print(result)

[1, 1, -1, 0]


To check what are the possible options of score in SentiStrenght go there: https://pypi.org/project/sentistrength/

In [2]:
import pandas as pd
from datetime import datetime
from tqdm import tqdm

data = pd.read_csv("tweets_only_eng_and_US_without_cleaning_2020-09-24.csv")
data = data[1:50000]

In [3]:
import numpy as np
import pandas as pd
import re
from spellchecker import SpellChecker
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import demoji
from bs4 import BeautifulSoup
demoji.download_codes()

# Initializes the spell checker, tokenizer and lammatizer.
check = SpellChecker()
tokenizer = RegexpTokenizer(r'\w+')
lemma = WordNetLemmatizer()

# Create a set of stopwords
stop_words = set(stopwords.words('english'))



# Cleaning functions

########################### FUNCTIONS ###################################
def correct_text(text):
# text needs to be a list of clean word tokens without other characters.
    misspelled = check.unknown(text)
    for word in misspelled:
        text[text.index(word)] = check.correction(word)
    return text

def lemmatize_text(text):
    return [lemma.lemmatize(word) for word in text]

# removing URL links (http or www pattern)
def rm_URL_lowcase(record: str) -> str:
    regex = r'https?://[^\s<>"]+|www\.[^\s<>"]+'
    sub_record = re.sub(regex, '', record)
    return sub_record

# removing hashtags and @
def rm_hashtags(record: str) -> str:
    sub_record = re.sub(r'@|#', ' ', record)
    return sub_record

# removing r"\n" 
def rm_trimwhitespaces(record: str) -> str:
    sub_record = strip(record)
    return sub_record

# removing punctuation
def rm_punctuation(record: str) -> str:
    s = re.sub(r'[^\w\s]', '', record)
    return s


# removing stopwords - i.e. the, a, an, he\
def rm_stopwords(record: str) -> str:
    words = list(record.split(' '))
    filtered_sentence = ' '.join([w for w in words if not w in stop_words])
    return filtered_sentence

# removeing emoji
def rm_emoji(record: str) -> str:
    plain = demoji.replace(record, " ")
    return plain

# Removing html coding
def rm_html(record: str) -> str:
    soup = BeautifulSoup(record, 'html.parser')
    plain = soup.get_text()
    return plain


# A function that extracts which words exist in a text based on a list of words to which we compare.
def word_feats(words):
    return dict([(word, True) for word in words])

... OK (Got response in 0.61 seconds)
Writing emoji data to C:\Users\PC\.demoji\codes.json ...
... OK


In [4]:
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

data = data.rename(columns = {'Unnamed: 0':'initial_index'})
data = data[['initial_index','text']]

data.loc[:, 'normalized'] = data['text'].apply(lambda x: re.sub(r'https?://[^\s<>"]+|www\.[^\s<>"]+', "", x))
print("Regex done at:", datetime.now())
data.loc[:, 'normalized'] = data['normalized'].progress_apply(lambda x: rm_html(x))
print("XTML decoding deleted at:", datetime.now())
#data.loc[:, 'normalized'] = data['normalized'].apply(lambda x: rm_newline(x))
#print("Newlines cleaned at:", datetime.now())
data.loc[:, 'normalized'] = data['normalized'].apply(lambda x: re.sub(r'@|#', ' ', x))
print("Hashtags done at:", datetime.now())
data.loc[:, 'normalized'] = data['normalized'].apply(lambda x: re.sub(r'[0-9]+', ' ', x))
print("Numbers replaced at:", datetime.now())
data.loc[:, 'normalized'] = data['normalized'].apply(lambda x: re.sub(r'\_', ' ', x))
print("Numbers replaced at:", datetime.now())

data.loc[:, 'normalized'] = data['normalized'].apply(lambda x: x.rstrip())
print("Whitespaces done at:", datetime.now())

C:\Users\PC\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Regex done at: 2020-09-24 11:55:05.058952




XTML decoding deleted at: 2020-09-24 11:55:18.117504
Hashtags done at: 2020-09-24 11:55:18.358360
Numbers replaced at: 2020-09-24 11:55:18.705165
Numbers replaced at: 2020-09-24 11:55:18.865073
Whitespaces done at: 2020-09-24 11:55:18.924041


In [9]:
data['normalized'][4]

' HillaryClinton you ARE the co-founder of ISIS, you crooked, evil, lying, witch. How can you live with yourself?'

In [10]:
data_with_sentiment = pd.read_csv('data_with_sentisentiment50k.csv')
data_with_sentiment = data_with_sentiment[['initial_index','text','normalized','sentiment','sentiment_to_naive']]
data_with_sentiment.head()

,initial_index,text,normalized,sentiment,sentiment_to_naive
0,4,#CNN #newday clear #Trump deliberately throwin...,CNN newday clear Trump deliberately throwin...,[1],pos
1,5,"@realDonaldTrump, you wouldn't recognize a lie...","realDonaldTrump, you wouldn't recognize a lie...",[1],pos
2,7,"""Kid, you know, suing someone? Thats the most ...","""Kid, you know, suing someone? Thats the most ...",[1],pos
3,8,@HillaryClinton you ARE the co-founder of ISIS...,HillaryClinton you ARE the co-founder of ISIS...,[-1],neg
4,10,@mike4193496 @realDonaldTrump I TOTALLY CONCUR...,mike4193496 realDonaldTrump I TOTALLY CONCUR...,[1],pos


In [11]:
### adding sentiment to current data
data_with_sentiment = data_with_sentiment[['initial_index','sentiment','sentiment_to_naive']]
data = pd.merge(left=data, right=data_with_sentiment, how='left', left_on='initial_index', right_on='initial_index', suffixes = ("","_senti"))
data.head()

,initial_index,text,normalized,sentiment,sentiment_to_naive
0,4,#CNN #newday clear #Trump deliberately throwin...,CNN newday clear Trump deliberately throwin...,[1],pos
1,5,"@realDonaldTrump, you wouldn't recognize a lie...","realDonaldTrump, you wouldn't recognize a lie...",[1],pos
2,7,"""Kid, you know, suing someone? Thats the most ...","""Kid, you know, suing someone? Thats the most ...",[1],pos
3,8,@HillaryClinton you ARE the co-founder of ISIS...,HillaryClinton you ARE the co-founder of ISIS...,[-1],neg
4,10,@mike4193496 @realDonaldTrump I TOTALLY CONCUR...,mike realDonaldTrump I TOTALLY CONCUR!! Thi...,[1],pos


In [ ]:
#data[['initial_index','normalized']].to_csv(r'C:\Users\PC\Desktop\UvA Master\Block 1\FDS\Assignment 1_ Politics (Twitter)\tweets_after_cleaning_with_sentiment.txt', header=True, index=None, sep=' ', mode='a')
print("Start sentiment", datetime.now())
data.loc[:,'sentiment'] = data['normalized'].progress_apply(lambda x: senti.getSentiment(x, score = 'binary'))
print("Sentiment done at:", datetime.now())
data[['initial_index','normalized']].to_csv('tweets_to_build_a_model_with_sentiment.csv')

data

In [ ]:
data.loc[:, 'sentiment_to_naive'] = data['sentiment'].apply(lambda x: 'pos' if x == [1] else 'neg')
data

In [12]:
data

,initial_index,text,normalized,sentiment,sentiment_to_naive
0,4,#CNN #newday clear #Trump deliberately throwin...,CNN newday clear Trump deliberately throwin...,[1],pos
1,5,"@realDonaldTrump, you wouldn't recognize a lie...","realDonaldTrump, you wouldn't recognize a lie...",[1],pos
2,7,"""Kid, you know, suing someone? Thats the most ...","""Kid, you know, suing someone? Thats the most ...",[1],pos
3,8,@HillaryClinton you ARE the co-founder of ISIS...,HillaryClinton you ARE the co-founder of ISIS...,[-1],neg
4,10,@mike4193496 @realDonaldTrump I TOTALLY CONCUR...,mike realDonaldTrump I TOTALLY CONCUR!! Thi...,[1],pos
...,...,...,...,...,...
49994,63498,@realDonaldTrump @nytimes where is your tax re...,realDonaldTrump nytimes where is your tax re...,[-1],neg
49995,63499,"@realDonaldTrump you said, ""you would bomb the...","realDonaldTrump you said, ""you would bomb the...",[-1],neg
49996,63501,"@mcuban At this point, I'm concerned for the s...","mcuban At this point, I'm concerned for the s...",[1],pos
49997,63502,"@realDonaldTrump yes you do stay on message, y...","realDonaldTrump yes you do stay on message, y...",[1],pos


In [13]:
# now we have data with a sentiment and we want to prepare them for bayes
data.loc[:, 'normalized'] = data['normalized'].apply(lambda x: rm_emoji(x))
print("Numbers replaced at:", datetime.now())
data.loc[:, 'normalized'] = data['normalized'].apply(lambda x: x.lower())
print("Lowered at:", datetime.now())
data.loc[:, 'normalized'] = data['normalized'].apply(lambda x: rm_punctuation(x))
print("Punctuation done at:", datetime.now())
data.loc[:, 'normalized'] = data['normalized'].apply(lambda x: rm_stopwords(x))
print("Stop words bye at:", datetime.now())


Numbers replaced at: 2020-09-24 11:57:09.777772
Lowered at: 2020-09-24 11:57:09.844737
Punctuation done at: 2020-09-24 11:57:10.183539
Stop words bye at: 2020-09-24 11:57:10.538345


In [14]:
data.head()

,initial_index,text,normalized,sentiment,sentiment_to_naive
0,4,#CNN #newday clear #Trump deliberately throwin...,cnn newday clear trump deliberately throwin...,[1],pos
1,5,"@realDonaldTrump, you wouldn't recognize a lie...",realdonaldtrump wouldnt recognize lie came mo...,[1],pos
2,7,"""Kid, you know, suing someone? Thats the most ...",kid know suing someone thats beautiful thing ...,[1],pos
3,8,@HillaryClinton you ARE the co-founder of ISIS...,hillaryclinton cofounder isis crooked evil ly...,[-1],neg
4,10,@mike4193496 @realDonaldTrump I TOTALLY CONCUR...,mike realdonaldtrump totally concur electio...,[1],pos


In [15]:
print("Start tokenization:", datetime.now())
data.loc[:, 'normalized'] = data['normalized'].apply(lambda x: tokenizer.tokenize(x))
print("Tokenized at:", datetime.now())
data.loc[:, 'normalized'] = data['normalized'].apply(lambda x: lemmatize_text(x))
print("Lemmatized at:", datetime.now())

# Removing all the rows that are empty in the text column after cleaning
#data = data.replace("", np.nan).replace([], np.nan).dropna()
data.head()

Start tokenization: 2020-09-24 11:57:20.850453
Tokenized at: 2020-09-24 11:57:21.438117
Lemmatized at: 2020-09-24 11:57:28.354174


,initial_index,text,normalized,sentiment,sentiment_to_naive
0,4,#CNN #newday clear #Trump deliberately throwin...,"[cnn, newday, clear, trump, deliberately, thro...",[1],pos
1,5,"@realDonaldTrump, you wouldn't recognize a lie...","[realdonaldtrump, wouldnt, recognize, lie, cam...",[1],pos
2,7,"""Kid, you know, suing someone? Thats the most ...","[kid, know, suing, someone, thats, beautiful, ...",[1],pos
3,8,@HillaryClinton you ARE the co-founder of ISIS...,"[hillaryclinton, cofounder, isi, crooked, evil...",[-1],neg
4,10,@mike4193496 @realDonaldTrump I TOTALLY CONCUR...,"[mike, realdonaldtrump, totally, concur, elect...",[1],pos


Now we need to prepare a column with pos and neg outcomes mapped out of our sentiment in order to put them for bayes.

In [17]:
data['normalized'][1]

['realdonaldtrump',
 'wouldnt',
 'recognize',
 'lie',
 'came',
 'mouth',
 'continually',
 'nevertrump']

In [22]:
data[['text','normalized','sentiment_to_naive']]
#data[data['sentiment_to_naive'] == 'pos']

,text,normalized,sentiment_to_naive
1,#CNN #newday clear #Trump deliberately throwin...,cnn newday clear trump deliberately throwin...,pos
2,"@realDonaldTrump, you wouldn't recognize a lie...",realdonaldtrump wouldnt recognize lie came mo...,pos
3,"""Kid, you know, suing someone? Thats the most ...",kid know suing someone thats beautiful thing 1...,pos
4,@HillaryClinton you ARE the co-founder of ISIS...,hillaryclinton cofounder isis crooked evil ly...,neg
5,@mike4193496 @realDonaldTrump I TOTALLY CONCUR...,mike4193496 realdonaldtrump totally concur e...,pos
...,...,...,...
49995,@realDonaldTrump @nytimes where is your tax re...,realdonaldtrump nytimes tax return,neg
49996,"@realDonaldTrump you said, ""you would bomb the...",realdonaldtrump said would bomb shit isil fuc...,neg
49997,"@mcuban At this point, I'm concerned for the s...",mcuban point im concerned safety hillaryclin...,pos
49998,"@realDonaldTrump yes you do stay on message, y...",realdonaldtrump yes stay message mentioned me...,pos


Now we need to put the words as a features with corresponsing sentiment information for them.

In [18]:
#data.to_csv("data_with_sentisentiment50k.csv")

Accuracy of our model build on a SentiStrenght corpus is equal to...
Now we want to map rest of the output to its sentiments.

### Checking Accuracy of a SentiStrenght

Take data with tweets that are manually labelled and check the accuracy of the classifier:

In [46]:
import numpy as np
import pandas as pd
import re
from spellchecker import SpellChecker
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import demoji
from bs4 import BeautifulSoup
#demoji.download_codes()

# Initializes the spell checker, tokenizer and lammatizer.
check = SpellChecker()
tokenizer = RegexpTokenizer(r'\w+')
lemma = WordNetLemmatizer()

# Create a set of stopwords
stop_words = set(stopwords.words('english'))

# Removing html coding
def rm_html(record: str) -> str:
    soup = BeautifulSoup(record, 'html.parser')
    plain = soup.get_text()
    return plain


test_politics_tweets = pd.read_excel("random_tweets_as_a_test_labeled.xlsx")
test_politics_tweets

,Unnamed: 0,text,sentiment
0,184569,@johnand2015 @wme98 @TrumpDynastyUSA @realDona...,pos
1,602960,@realDonaldTrump AMERICANS WANT A CHANGE,pos
2,412384,"@realDonaldTrump is all in, I'm all in. Histor...",pos
3,423577,@JoeFreedomLove @FreeBeacon \nThat's Because T...,neg
4,647595,Not pneumonia symptoms: Med guideline requires...,neg
...,...,...,...
294,495873,@USA_FREEDOM_NOW @klmstlouis @realDonaldTrump ...,neg
295,615525,"#Hillary is corrupting, lying to our young peo...",neg
296,325430,@andylefko @KeithOlbermann @realDonaldTrump @S...,neg
297,460252,@amjoyshow #AMJoy #Trump is ripping off part o...,neg


In [47]:
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

test_politics_tweets.loc[:, 'cleaned_for_senti'] = test_politics_tweets['text'].apply(lambda x: re.sub(r'https?://[^\s<>"]+|www\.[^\s<>"]+', "", x))
print("Regex done at:", datetime.now())
test_politics_tweets.loc[:, 'cleaned_for_senti'] = test_politics_tweets['cleaned_for_senti'].progress_apply(lambda x: rm_html(x))
print("XTML decoding deleted at:", datetime.now())
#data.loc[:, 'normalized'] = data['normalized'].apply(lambda x: rm_newline(x))
#print("Newlines cleaned at:", datetime.now())
test_politics_tweets.loc[:, 'cleaned_for_senti'] = test_politics_tweets['cleaned_for_senti'].apply(lambda x: re.sub(r'@|#', ' ', x))
print("Hashtags done at:", datetime.now())
test_politics_tweets.loc[:, 'cleaned_for_senti'] = test_politics_tweets['cleaned_for_senti'].apply(lambda x: re.sub(r'[0-9]+', ' ', x))
print("Numbers replaced at:", datetime.now())
test_politics_tweets.loc[:, 'cleaned_for_senti'] = test_politics_tweets['cleaned_for_senti'].apply(lambda x: re.sub(r'\_', ' ', x))
print("Numbers replaced at:", datetime.now())
test_politics_tweets.loc[:, 'cleaned_for_senti'] = test_politics_tweets['cleaned_for_senti'].apply(lambda x: x.rstrip())
print("Whitespaces done at:", datetime.now())

Regex done at: 2020-09-24 15:47:54.298729



XTML decoding deleted at: 2020-09-24 15:47:54.599557
Hashtags done at: 2020-09-24 15:47:54.613549
Numbers replaced at: 2020-09-24 15:47:54.628541
Numbers replaced at: 2020-09-24 15:47:54.639535
Whitespaces done at: 2020-09-24 15:47:54.649529


In [48]:
test_politics_tweets

,Unnamed: 0,text,sentiment,cleaned_for_senti
0,184569,@johnand2015 @wme98 @TrumpDynastyUSA @realDona...,pos,johnand wme TrumpDynastyUSA realDonaldTr...
1,602960,@realDonaldTrump AMERICANS WANT A CHANGE,pos,realDonaldTrump AMERICANS WANT A CHANGE
2,412384,"@realDonaldTrump is all in, I'm all in. Histor...",pos,"realDonaldTrump is all in, I'm all in. Histor..."
3,423577,@JoeFreedomLove @FreeBeacon \nThat's Because T...,neg,JoeFreedomLove FreeBeacon \nThat's Because T...
4,647595,Not pneumonia symptoms: Med guideline requires...,neg,Not pneumonia symptoms: Med guideline requires...
...,...,...,...,...
294,495873,@USA_FREEDOM_NOW @klmstlouis @realDonaldTrump ...,neg,USA FREEDOM NOW klmstlouis realDonaldTrump ...
295,615525,"#Hillary is corrupting, lying to our young peo...",neg,"Hillary is corrupting, lying to our young peo..."
296,325430,@andylefko @KeithOlbermann @realDonaldTrump @S...,neg,andylefko KeithOlbermann realDonaldTrump S...
297,460252,@amjoyshow #AMJoy #Trump is ripping off part o...,neg,amjoyshow AMJoy Trump is ripping off part o...


In [49]:
test_politics_tweets.loc[:,'sentistrenght_sentiment'] = test_politics_tweets['cleaned_for_senti'].progress_apply(lambda x: senti.getSentiment(x, score = 'binary'))
print("Sentiment done at:", datetime.now())
test_politics_tweets.head()


Sentiment done at: 2020-09-24 15:49:29.426581


,Unnamed: 0,text,sentiment,cleaned_for_senti,sentistrenght_sentiment
0,184569,@johnand2015 @wme98 @TrumpDynastyUSA @realDona...,pos,johnand wme TrumpDynastyUSA realDonaldTr...,[1]
1,602960,@realDonaldTrump AMERICANS WANT A CHANGE,pos,realDonaldTrump AMERICANS WANT A CHANGE,[1]
2,412384,"@realDonaldTrump is all in, I'm all in. Histor...",pos,"realDonaldTrump is all in, I'm all in. Histor...",[1]
3,423577,@JoeFreedomLove @FreeBeacon \nThat's Because T...,neg,JoeFreedomLove FreeBeacon \nThat's Because T...,[1]
4,647595,Not pneumonia symptoms: Med guideline requires...,neg,Not pneumonia symptoms: Med guideline requires...,[1]


In [50]:
test_politics_tweets.loc[:, 'sentistrenght_sentiment_translated'] = test_politics_tweets['sentistrenght_sentiment'].apply(lambda x: 'pos' if x == [1] else 'neg')
test_politics_tweets

,Unnamed: 0,text,sentiment,cleaned_for_senti,sentistrenght_sentiment,sentistrenght_sentiment_translated
0,184569,@johnand2015 @wme98 @TrumpDynastyUSA @realDona...,pos,johnand wme TrumpDynastyUSA realDonaldTr...,[1],pos
1,602960,@realDonaldTrump AMERICANS WANT A CHANGE,pos,realDonaldTrump AMERICANS WANT A CHANGE,[1],pos
2,412384,"@realDonaldTrump is all in, I'm all in. Histor...",pos,"realDonaldTrump is all in, I'm all in. Histor...",[1],pos
3,423577,@JoeFreedomLove @FreeBeacon \nThat's Because T...,neg,JoeFreedomLove FreeBeacon \nThat's Because T...,[1],pos
4,647595,Not pneumonia symptoms: Med guideline requires...,neg,Not pneumonia symptoms: Med guideline requires...,[1],pos
...,...,...,...,...,...,...
294,495873,@USA_FREEDOM_NOW @klmstlouis @realDonaldTrump ...,neg,USA FREEDOM NOW klmstlouis realDonaldTrump ...,[-1],neg
295,615525,"#Hillary is corrupting, lying to our young peo...",neg,"Hillary is corrupting, lying to our young peo...",[-1],neg
296,325430,@andylefko @KeithOlbermann @realDonaldTrump @S...,neg,andylefko KeithOlbermann realDonaldTrump S...,[-1],neg
297,460252,@amjoyshow #AMJoy #Trump is ripping off part o...,neg,amjoyshow AMJoy Trump is ripping off part o...,[1],pos


In [51]:
test_politics_tweets.loc[:, 'if_correct_sentiment'] = np.where(test_politics_tweets['sentiment'] == test_politics_tweets['sentistrenght_sentiment_translated'], 'yes', 'no')
test_politics_tweets

,Unnamed: 0,text,sentiment,cleaned_for_senti,sentistrenght_sentiment,sentistrenght_sentiment_translated,if_correct_sentiment
0,184569,@johnand2015 @wme98 @TrumpDynastyUSA @realDona...,pos,johnand wme TrumpDynastyUSA realDonaldTr...,[1],pos,yes
1,602960,@realDonaldTrump AMERICANS WANT A CHANGE,pos,realDonaldTrump AMERICANS WANT A CHANGE,[1],pos,yes
2,412384,"@realDonaldTrump is all in, I'm all in. Histor...",pos,"realDonaldTrump is all in, I'm all in. Histor...",[1],pos,yes
3,423577,@JoeFreedomLove @FreeBeacon \nThat's Because T...,neg,JoeFreedomLove FreeBeacon \nThat's Because T...,[1],pos,no
4,647595,Not pneumonia symptoms: Med guideline requires...,neg,Not pneumonia symptoms: Med guideline requires...,[1],pos,no
...,...,...,...,...,...,...,...
294,495873,@USA_FREEDOM_NOW @klmstlouis @realDonaldTrump ...,neg,USA FREEDOM NOW klmstlouis realDonaldTrump ...,[-1],neg,yes
295,615525,"#Hillary is corrupting, lying to our young peo...",neg,"Hillary is corrupting, lying to our young peo...",[-1],neg,yes
296,325430,@andylefko @KeithOlbermann @realDonaldTrump @S...,neg,andylefko KeithOlbermann realDonaldTrump S...,[-1],neg,yes
297,460252,@amjoyshow #AMJoy #Trump is ripping off part o...,neg,amjoyshow AMJoy Trump is ripping off part o...,[1],pos,no


In [56]:
test_politics_tweets['if_correct_sentiment'].value_counts()

yes    167
no     132
Name: if_correct_sentiment, dtype: int64

In [52]:
test_politics_tweets.groupby(['sentiment','if_correct_sentiment']).size()

sentiment  if_correct_sentiment
neg        no                      116
           yes                     100
pos        no                       16
           yes                      67
dtype: int64

accuracy for positive tweets 67/(67+16) which is around 80,72%

accuracy for negative tweets 100/216 which is around 46,30%